<a href="https://colab.research.google.com/github/ShreyaSarkar1234/DEEP_LEARNING/blob/main/q4neww.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Sample data
texts = [
    "I love this product",
    "This is the worst thing ever",
    "Not sure how I feel",
    "Absolutely fantastic experience",
    "Terrible service",
    "It’s okay, not bad",
    "I am very happy",
    "I hate it",
    "Meh, nothing special",
]

labels = [2, 0, 1, 2, 0, 1, 2, 0, 1]  # 0=Neg, 1=Neutral, 2=Pos

# Tokenization
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=10, padding='post')

# Convert to tensors
X = torch.tensor(padded, dtype=torch.long)
y = torch.tensor(labels, dtype=torch.long)

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = SentimentDataset(X, y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Define Model
class SentimentModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SentimentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        out = self.fc(hidden[-1])
        return out

# Hyperparameters
vocab_size = 1000
embedding_dim = 16
hidden_dim = 32
output_dim = 3

# Model, loss, optimizer
model = SentimentModel(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
model.train()
for epoch in range(10):
    total_loss = 0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Prediction
model.eval()
user_input = input("Enter your sentence: ")
user_seq = tokenizer.texts_to_sequences([user_input])
user_pad = pad_sequences(user_seq, maxlen=10, padding='post')
user_tensor = torch.tensor(user_pad, dtype=torch.long)

with torch.no_grad():
    output = model(user_tensor)
    pred_label = torch.argmax(output, dim=1).item()

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
print(f"Predicted Sentiment: {label_map[pred_label]}")


Epoch 1, Loss: 5.5274
Epoch 2, Loss: 5.4954
Epoch 3, Loss: 5.5287
Epoch 4, Loss: 5.5679
Epoch 5, Loss: 5.5219
Epoch 6, Loss: 5.5121
Epoch 7, Loss: 5.4687
Epoch 8, Loss: 5.5064
Epoch 9, Loss: 5.5379
Epoch 10, Loss: 5.4896
Enter your sentence: I LOVE TO EAT PANIPURI
Predicted Sentiment: Positive
